**Qrisp (Quantum RISP) is a high-level, open-source programming framework for quantum computing**  
Notebook, practice Carlos Araque

In order to use qrisp together with IQM devices, please make sure to install the latest version of qrisp.

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install --no-cache --force-reinstall --upgrade "iqm-client>=32.1.1, <33.0" "qrisp[iqm]"

# Here are practical Qrisp code examples demonstrating its core features and quantum algorithms:

Carlos Araque  
Quantum Developer  
Melbourne, Australia

Initial Source:
https://qrisp.eu/index.html


**Basic Quantum Variable Creation**

In [1]:
from qrisp import QuantumVariable, h, z, cx

# Create a quantum variable with 5 qubits
qv = QuantumVariable(5)

# Apply gates
h(qv[0])  # Hadamard on first qubit
z(qv)     # Z gate on all qubits
cx(qv[0], qv[3])  # CNOT between qubits 0 and 3

# View the circuit
print(qv.qs)

# Measure and see results
print(qv)


/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.0, so it will not be used.
  warnings.warn(


QuantumCircuit:
---------------
      ┌───┐┌───┐     
qv.0: ┤ H ├┤ Z ├──■──
      ├───┤└───┘  │  
qv.1: ┤ Z ├───────┼──
      ├───┤       │  
qv.2: ┤ Z ├───────┼──
      ├───┤     ┌─┴─┐
qv.3: ┤ Z ├─────┤ X ├
      ├───┤     └───┘
qv.4: ┤ Z ├──────────
      └───┘          
Live QuantumVariables:
----------------------
QuantumVariable qv
{'00000': 0.5, '10010': 0.5}


**GHZ State Creation**

In [2]:
from qrisp import QuantumVariable, h, cx

def create_ghz_state(n_qubits=3):
    qv = QuantumVariable(n_qubits)
    h(qv[0])  # Hadamard on first qubit

    # Entangle all qubits
    for i in range(n_qubits - 1):
        cx(qv[i], qv[i + 1])

    return qv

# Create and measure a 5-qubit GHZ state
ghz = create_ghz_state(5)
print(ghz)
# Output: {'00000': 0.5, '11111': 0.5}


{'00000': 0.5, '11111': 0.5}


GHZ in Qrisp (simple version)

In [3]:
from qrisp import QuantumVariable, h, cx

q = QuantumVariable(10)
h(q[0])
for i in range(9):
    cx(q[i], q[i+1])


**Quantum Characters and Entanglement**

In [4]:
from qrisp import QuantumChar, QuantumVariable, cx

# Create quantum character
qch = QuantumChar()
qch[:] = "e"  # Initialize to 'e'
print(qch)  # {'e': 1.0}

# Entangle with another quantum variable
qv = QuantumVariable(1)
cx(qv[0], qch[0])
print(qch)  # {'e': 0.5, 'f': 0.5}


{'e': 1.0}
{'e': 1.0}


**Grover's Algorithm - Solving Quadratic Equations**

In [5]:
from qrisp import QuantumFloat, auto_uncompute, z, h
from qrisp.grover import diffuser
from math import pi

# Oracle to find sqrt(0.25)
@auto_uncompute
def sqrt_oracle(qf):
    temp_qbool = qf * qf == 0.25
    z(temp_qbool)

# Execute Grover's algorithm
qf = QuantumFloat(3, -1, signed=True)
n = qf.size
iterations = int(0.25 * pi * (2**n / 2) ** 0.5)

h(qf)  # Create superposition
for i in range(iterations):
    sqrt_oracle(qf)
    diffuser(qf)

print(qf)
# Output: {0.5: 0.47, -0.5: 0.47, ...}


{0.5: 0.47263164210147385, -0.5: 0.47263164210147385, 0.0: 0.003909765414075155, 1.0: 0.003909765414075155, 1.5: 0.003909765414075155, 2.0: 0.003909765414075155, 2.5: 0.003909765414075155, 3.0: 0.003909765414075155, 3.5: 0.003909765414075155, -4.0: 0.003909765414075155, -3.5: 0.003909765414075155, -3.0: 0.003909765414075155, -2.5: 0.003909765414075155, -2.0: 0.003909765414075155, -1.5: 0.003909765414075155, -1.0: 0.003909765414075155}


Grover's Algorithm - Finding Specific States

In [6]:
from qrisp import QuantumFloat
from qrisp.grover import tag_state, grovers_alg

# Create two QuantumFloats
qf_list = [QuantumFloat(2, signed=True),
           QuantumFloat(2, signed=True)]

# Oracle that tags states (-3, 2)
def test_oracle(qf_list):
    tag_dic = {qf_list[0]: -3, qf_list[1]: 2}
    tag_state(tag_dic)

# Run Grover's algorithm
grovers_alg(qf_list, test_oracle)

# Measure both variables
from qrisp.misc import multi_measurement
print(multi_measurement(qf_list))
# Output: {(-3, 2): 0.9966, ...}


{(-3, 2): 0.9968491807870038, (0, 0): 5.001300338087899e-05, (0, 1): 5.001300338087899e-05, (0, 2): 5.001300338087899e-05, (0, 3): 5.001300338087899e-05, (0, -4): 5.001300338087899e-05, (0, -3): 5.001300338087899e-05, (0, -2): 5.001300338087899e-05, (0, -1): 5.001300338087899e-05, (1, 0): 5.001300338087899e-05, (1, 1): 5.001300338087899e-05, (1, 2): 5.001300338087899e-05, (1, 3): 5.001300338087899e-05, (1, -4): 5.001300338087899e-05, (1, -3): 5.001300338087899e-05, (1, -2): 5.001300338087899e-05, (1, -1): 5.001300338087899e-05, (2, 0): 5.001300338087899e-05, (2, 1): 5.001300338087899e-05, (2, 2): 5.001300338087899e-05, (2, 3): 5.001300338087899e-05, (2, -4): 5.001300338087899e-05, (2, -3): 5.001300338087899e-05, (2, -2): 5.001300338087899e-05, (2, -1): 5.001300338087899e-05, (3, 0): 5.001300338087899e-05, (3, 1): 5.001300338087899e-05, (3, 2): 5.001300338087899e-05, (3, 3): 5.001300338087899e-05, (3, -4): 5.001300338087899e-05, (3, -3): 5.001300338087899e-05, (3, -2): 5.001300338087899

**Quantum Phase Estimation**

In [7]:
from qrisp import QuantumFloat, QuantumVariable, control, QFT, h, p
import numpy as np

def QPE(psi, U, precision):
    res = QuantumFloat(precision, -precision)
    h(res)  # Create superposition

    # Controlled-U operations
    for i in range(precision):
        with control(res[i]):
            for j in range(2**i):
                U(psi)

    return QFT(res, inv=True)  # Inverse QFT

# Test with phase gates
def U(psi):
    phi_1 = 0.5
    phi_2 = 0.125
    p(phi_1 * 2 * np.pi, psi[0])
    p(phi_2 * 2 * np.pi, psi[1])

psi = QuantumVariable(2)
h(psi)
res = QPE(psi, U, 3)

# Measure both variables together
from qrisp.misc import multi_measurement
print(multi_measurement([psi, res]))
# Output: {('00', 0.0): 0.25, ('10', 0.5): 0.25, ('01', 0.125): 0.25, ('11', 0.625): 0.25}


{('00', 0.0): 0.25, ('10', 0.5): 0.25, ('01', 0.125): 0.25, ('11', 0.625): 0.25}


**Create a simple Qrisp script to build a GHZ **

In [8]:
from qrisp import QuantumVariable, h, cx

def create_ghz_state(n_qubits=3):
    """
    Create a GHZ (Greenberger-Horne-Zeilinger) state.
    GHZ state: |000...0⟩ + |111...1⟩ (equal superposition of all 0s and all 1s)
    """
    # Create a quantum variable with n_qubits
    qv = QuantumVariable(n_qubits)

    # Apply Hadamard to the first qubit to create superposition
    h(qv[0])

    # Apply CNOT gates to entangle all qubits
    for i in range(n_qubits - 1):
        cx(qv[i], qv[i + 1])

    return qv

# Create a 3-qubit GHZ state
ghz = create_ghz_state(3)

# View the quantum circuit
print("Quantum Circuit:")
print(ghz.qs)

# Measure the state and get results (default: 1000 shots)
results = ghz.get_measurement(shots=1000)
print("\nMeasurement Results:")
print(results)

# Expected output: {'000': ~0.5, '111': ~0.5}
# This shows the characteristic GHZ state behavior

# Optional: View the statevector
print("\nStatevector:")
print(ghz.qs.statevector())


Quantum Circuit:
QuantumCircuit:
---------------
      ┌───┐          
qv.0: ┤ H ├──■───────
      └───┘┌─┴─┐     
qv.1: ─────┤ X ├──■──
           └───┘┌─┴─┐
qv.2: ──────────┤ X ├
                └───┘
Live QuantumVariables:
----------------------
QuantumVariable qv

Measurement Results:
{'000': 0.5, '111': 0.5}

Statevector:
sqrt(2)*(|000> + |111>)/2


I use here by Ca_API_Token

In [ ]:
api_token = input("Enter your IQM Resonance API token: ")
#Alrady executed

**IQMBackend**  
plus.get_measurement(backend=quantum_computer)

In [10]:
from qrisp import QuantumVariable, h, x, cx
from qrisp.interface import IQMBackend

# Replace with your actual IQM Resonance API token
# api_token = "YOUR_IQM_RESONANCE_TOKEN"

# Connect to IQM's quantum computer (Sirius or other available devices)
# Check IQM Resonance Dashboard for current available devices
quantum_computer = IQMBackend(
    api_token=api_token,
    device_instance="sirius"  # Options: "sirius", "garnet", "crystal", etc.
)

print("Connected to IQM Quantum Computer: Sirius\n")

# Task 2.1 Examples: Create basic quantum states
# State |0⟩
zero = QuantumVariable(1)
print("Created |0⟩ state")

# State |1⟩
one = QuantumVariable(1)
x(one)  # Apply X gate to flip |0⟩ to |1⟩
print("Created |1⟩ state")

# State |+⟩ = (|0⟩ + |1⟩)/√2
plus = QuantumVariable(1)
h(plus)  # Apply Hadamard gate
print("Created |+⟩ state")

# State |-⟩ = (|0⟩ - |1⟩)/√2
minus = QuantumVariable(1)
x(minus)
h(minus)  # X then Hadamard creates |-⟩
print("Created |-⟩ state\n")

# Task 3.1: Run on IQM quantum hardware
print("=" * 50)
print("Running on IQM Real Quantum Hardware")
print("=" * 50)

# Test |0⟩ state on real hardware
print("\nMeasuring |0⟩ state:")
result_zero = zero.get_measurement(backend=quantum_computer, shots=1000)
print(result_zero)

# Test |1⟩ state on real hardware
print("\nMeasuring |1⟩ state:")
result_one = one.get_measurement(backend=quantum_computer, shots=1000)
print(result_one)

# Test |+⟩ state on real hardware
print("\nMeasuring |+⟩ state:")
result_plus = plus.get_measurement(backend=quantum_computer, shots=1000)
print(result_plus)

# Test |-⟩ state on real hardware
print("\nMeasuring |-⟩ state:")
result_minus = minus.get_measurement(backend=quantum_computer, shots=1000)
print(result_minus)

# Bonus: Create and test an entangled Bell state
print("\n" + "=" * 50)
print("Bonus: Bell State on Real Hardware")
print("=" * 50)

bell = QuantumVariable(2)
h(bell[0])
cx(bell[0], bell[1])
print("\nMeasuring Bell state |Φ+⟩:")
result_bell = bell.get_measurement(backend=quantum_computer, shots=1000)
print(result_bell)
print("\nExpected: {'00': ~0.5, '11': ~0.5} (showing entanglement)")


Connected to IQM Quantum Computer: Sirius

Created |0⟩ state
Created |1⟩ state
Created |+⟩ state
Created |-⟩ state

Running on IQM Real Quantum Hardware

Measuring |0⟩ state:
{'0': 0.996, '1': 0.004}

Measuring |1⟩ state:
{'1': 0.982, '0': 0.018}

Measuring |+⟩ state:
{'1': 0.519, '0': 0.481}

Measuring |-⟩ state:
{'0': 0.501, '1': 0.499}

Bonus: Bell State on Real Hardware

Measuring Bell state |Φ+⟩:


Exception: Could not convert Qiskit operation move to Qrisp